In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score

In [6]:
from tqdm import tqdm

In [7]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Google 相册',
 'matlab源程序',
 '破解软件',
 '一体化数值仿真.docx',
 '2D40.jpg',
 'OpenFOAM 拉格朗日库深度解析 | OpenFOAM 成长之路.pdf',
 'Colab Notebooks',
 'fastai',
 '无标题文档.gdoc',
 'train_df_feat.npy',
 'test_df_feat.npy',
 'train_labeled_extend_with_test.csv',
 'model.pt']

## Module

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

In [9]:
from IPython.display import display
class FocalLoss(nn.Module):

    def __init__(self, weight=None, reduction='mean', gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = torch.nn.CrossEntropyLoss(weight=weight, reduction=reduction)

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [10]:
class CausalConv1d(nn.Module):
    """
    Input and output sizes will be the same.
    """
    def __init__(self, in_size, out_size, kernel_size, dilation=1,stride=1):
        super(CausalConv1d, self).__init__()
        self.pad = (kernel_size-1) // 2 * dilation
        self.conv1 = nn.Conv1d(in_size, out_size, kernel_size, padding=self.pad, stride=stride,dilation=dilation)

    def forward(self, x):
        x = self.conv1(x)
        return x

In [11]:
class ResidualLayer(nn.Module):    
    def __init__(self, residual_size, skip_size, dilation):
        super(ResidualLayer, self).__init__()
        self.conv_filter = CausalConv1d(residual_size, residual_size,
                                         kernel_size=3, dilation=dilation)
        self.conv_gate = CausalConv1d(residual_size, residual_size,
                                         kernel_size=3, dilation=dilation)        
        self.resconv1_1 = nn.Conv1d(residual_size, residual_size, kernel_size=1)
        self.skipconv1_1 = nn.Conv1d(residual_size, skip_size, kernel_size=1)
        
   
    def forward(self, x):
        conv_filter = self.conv_filter(x)
        conv_gate = self.conv_gate(x)  
        fx = F.tanh(conv_filter) * F.sigmoid(conv_gate)
        fx = self.resconv1_1(fx) 
        skip = self.skipconv1_1(fx) 
        residual = fx + x  
        #residual=[batch,residual_size,seq_len]  skip=[batch,skip_size,seq_len]
        return skip, residual

In [12]:
class DilatedStack(nn.Module):
    def __init__(self, residual_size, skip_size, dilation_depth):
        super(DilatedStack, self).__init__()
        residual_stack = [ResidualLayer(residual_size, skip_size, 2**layer)
                         for layer in range(dilation_depth)]
        self.residual_stack = nn.ModuleList(residual_stack)
        
    def forward(self, x):
        skips = []
        for layer in self.residual_stack:
            skip, x = layer(x)
            skips.append(skip.unsqueeze(0))
            #skip =[1,batch,skip_size,seq_len]
        return torch.cat(skips, dim=0), x  # [layers,batch,skip_size,seq_len]

In [13]:
class WaveNet(nn.Module):

    def __init__(self,input_size,out_size,num_class, residual_size, skip_size, dilation_cycles, dilation_depth):

        super(WaveNet, self).__init__()

        self.input_conv = CausalConv1d(input_size,residual_size, kernel_size=3)        

        self.dilated_stacks = nn.ModuleList(

            [DilatedStack(residual_size, skip_size, dilation_depth)

             for cycle in range(dilation_cycles)]

        )

        self.convout_1 = nn.Conv1d(skip_size, skip_size, kernel_size=1)
        self.finial=nn.Linear(skip_size,num_calss)

    def forward(self, x):
       # input=[batch,seq_len,]

        x = x.permute(0,2,1)# [batch,input_feature_dim, seq_len]

        x = self.input_conv(x) # [batch,residual_size, seq_len]             

        skip_connections = []

        for cycle in self.dilated_stacks:

            skips, x = cycle(x)             
            skip_connections.append(skips)

        ## skip_connection=[total_layers,batch,skip_size,seq_len]
        skip_connections = torch.cat(skip_connections, dim=0)        

        # gather all output skip connections to generate output, discard last residual output

        out = skip_connections.sum(dim=0) # [batch,skip_size,seq_len]

        out = F.relu(out)
        out = self.convout_1(out) # [batch,out_size,seq_len]
        
        out=torch.mean(out,dim=2)#=[128,256,100]

        out=self.finial(out) #[128,256,]= [128,10]
        return out     

In [14]:
def initialize_model(learning_rate,input_size,out_size,num_class,residual_size,skip_size,dilation_cycles, dilation_depth,device):
    torch.manual_seed(42)
    
    model=WaveNet(input_size,out_size,num_class,residual_size,skip_size,dilation_cycles, dilation_depth)
    model.to(device)

    criterion = FocalLoss(gamma=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  patience=100, factor =0.5 ,min_lr=1e-7, eps=1e-08)
    return model, criterion, optimizer, scheduler

In [61]:
from fastprogress import master_bar, progress_bar
def train_net(train_loader,val_loader,num_epochs,model,criterion,optimizer,scheduler,patience,verbose,user_define_score,device):
    valid_loss_min=np.Inf
    patience=patience
    current_epoch=0
    stop=False
    num_epochs=num_epochs
    training_loss=[]
    for epoch in progress_bar(range(num_epochs)): 
        train_loss=[]
        train_score=[]
        model.train()
        for batch_i,(data,target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output=model(data)
            loss=criterion(output,target.long())
            train_loss.append(loss.item())
            a=target.data.cpu().numpy()
            b=torch.argmax(F.softmax(output,dim=1),dim=1).detach().cpu().numpy()
            train_score.append(user_define_score(a,b,average='macro'))
            
            loss.backward()
            optimizer.step()

        val_loss=[]
        val_score=[]
        for batch_i,(data,target) in enumerate(val_loader):
            data,target=data.to(device),target.to(device)
            output=model(data)
            loss =criterion(output,target.long())
            val_loss.append(loss.item())
            a=target.data.cpu().numpy()
            b=torch.argmax(F.softmax(output,dim=1),dim=1).detach().cpu().numpy()
            val_score.append(user_define_score(a,b,average='macro'))


        ##if epoch%2 ==0 and verbose:
        print("{0},train loss {1},vllid loss {2}".format(epoch,np.mean(train_loss),np.mean(val_loss)))
          #print(f'Epoch {epoch},train loss:{np.mean(train_loss):.4f,valid loss:{np.mean(val_loss):.4f}}')
        scheduler.step(np.mean(val_loss))


        valid_loss=np.mean(val_loss)
        if valid_loss<=valid_loss_min:
            torch.save(model.state_dict(),"model.pt")
            valid_loss_min=valid_loss
            cureent_epoch=0
        
        if valid_loss>valid_loss_min:
            current_epoch+=1;
            if current_epoch>patience:
                print("stropping training")
                stop=True
                break
        if stop:
            break 


    checkpoint=torch.load("model.pt")
    model.load_state_dict(checkpoint)

    return model

In [53]:
def train_net_folds(X,y,test,flods,num_epochs,batch_size,patience,verbose,user_define_score,learning_rate,
                   input_size,out_size,num_class,residual_size,skip_size,dilation_cycles, dilation_depth,device):

    prediction = []
    scores = []
    for flod_n,(train_index,valid_index) in enumerate(flods.split(X,y)):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = np.array(y)[train_index], np.array(y)[valid_index]
        train_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
        val_set = torch.utils.data.TensorDataset(torch.FloatTensor(X_valid), torch.FloatTensor(y_valid))

        train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set,batch_size=batch_size)
        
        model, criterion, optimizer, scheduler = initialize_model(learning_rate,input_size,out_size,num_class,residual_size,skip_size,dilation_cycles, dilation_depth,device)
        model = train_net(train_loader, val_loader,num_epochs,model, criterion, optimizer, scheduler, patience,verbose,user_define_score,device)
        
        y_pred_valid = []
        for batch_i, (data, target) in enumerate(val_loader):
            data, target = data.to(device), target.to(device)
            p = model(data)
            pred = torch.argmax(F.softmax(p,dim=1),dim=1).detach().cpu().numpy()
            y_pred_valid.extend(pred)
        sc=user_define_score(y_valid, np.array(y_pred_valid),average='macro') 
        scores.append(sc)
        print("valid_score is{}".format(sc))
        ## predict 
        test=torch.FloatTensor(test)
        test_loader= torch.utils.data.DataLoader(test,batch_size=batch_size,shuffle=False)

        y_pred= []
        for batch_i,data in enumerate(test_loader):
            data= data.to(device)
            p = model(data)
            pred =F.softmax(p,dim=1).detach().cpu().numpy()
            y_pred.extend(pred)

        prediction.append(y_pred)

    prediction=np.sum(prediction,axis=0)/len(prediction)
    score=np.mean(scores)

    print('--' * 50)
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    print('--' * 50)
    print(prediction.shape)
    
    return prediction,score

In [17]:
!pwd

/content/drive/My Drive


In [18]:
train_feat=np.load("train_df_feat.npy")

In [19]:
test_feat=np.load("test_df_feat.npy")

In [20]:
test_feat[0]

array([ 0.0293984 ,  0.00309613,  0.04164624, -0.06482507, -0.12415203,
        0.06881723, -0.11917498,  0.19741213, -0.08306552, -0.02157832,
       -0.05257238,  0.03810216, -0.05991602,  0.20648812,  0.10207941,
        0.0265471 ,  0.16867273, -0.07933275, -0.01322009, -0.04931647,
        0.02572281, -0.03265868, -0.15895757, -0.06183881, -0.11126047,
       -0.11303195,  0.08269095, -0.234303  ,  0.04077412,  0.02167152,
        0.23450379, -0.02913769, -0.07981101, -0.11929078, -0.00844684,
        0.08854344, -0.0299254 ,  0.06171508, -0.1125944 ,  0.01242304,
       -0.06977975, -0.1811374 ,  0.01865262, -0.03779281, -0.0566307 ,
        0.07765503,  0.04205372, -0.12768736,  0.11262918,  0.1161378 ,
        0.14323595, -0.09277142, -0.1586106 ,  0.01528   , -0.17140804,
       -0.22404848,  0.07873732, -0.09596794,  0.06750242, -0.07185113,
       -0.09187223, -0.0332422 ,  0.01409989, -0.03288124, -0.06557307,
       -0.07158167,  0.03979059,  0.0268771 ,  0.16183574,  0.11

In [21]:
train_df=pd.read_csv("train_labeled_extend_with_test.csv")

In [22]:
train_df.head()

,label_ft,num_label,text
0,__label__3,3,453 27186 74256 2235 27 3346 109827 11887 4990...
1,__label__2,2,24359 267863 5111 46 913 267 39 267 81 28 157 ...
2,__label__3,3,2741 220 44 28 1 351 14742 2459 8 273 6313 413...
3,__label__5,5,10564 5360 27 11660 108490 29159 8273 1 16369 ...
4,__label__3,3,282 10564 14810 4279 607 44666 8702 126 90069 ...


In [23]:
X=[]
for item in train_feat:
    it=np.array(item.flatten())
    it=it.reshape(len(it),1)
    X.append(it)
X_test=[]
for item in test_feat:
    it=np.array(item.flatten())
    it=it.reshape(len(it),1)
    X_test.append(it)
y=np.array(train_df["num_label"])
X=np.array(X)
y=np.array(y)


In [24]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [63]:
user_error=f1_score
flods=StratifiedKFold(n_splits=5, random_state=7,shuffle=True) 
num_epochs=10
batch_size=128
patience=500
verbose=False
user_define_score=user_error
learning_rate=1e-3
input_size=1
out_size=1
num_calss=10
residual_size=128
skip_size=128
dilation_cycles=2
dilation_depth=3
predcit_results=[]
score=[]
predcit_results,score=train_net_folds(X,y,X_test,flods,num_epochs,batch_size,patience,verbose,user_define_score,learning_rate,
                   input_size,out_size,num_calss,residual_size,skip_size,dilation_cycles, dilation_depth,device)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0,train loss 2.0643786863496683,vllid loss 2.0597791684571134
1,train loss 1.8795890641019672,vllid loss 1.3314674044809034
2,train loss 0.5039821517355038,vllid loss 0.24242045473988338
3,train loss 0.1851551365257916,vllid loss 0.24848382938052377
4,train loss 0.1305857094110183,vllid loss 0.28080822100802777
5,train loss 0.10625628552166118,vllid loss 0.13428846842819644
6,train loss 0.0948733228617762,vllid loss 0.13600861749321383
7,train loss 0.08768489028545563,vllid loss 0.15506862846743916
8,train loss 0.07750566753576386,vllid loss 0.1537049678655001
9,train loss 0.06901399251023393,vllid loss 0.12088127929206577
valid_score is0.8986566673669092


0,train loss 1.524720194847436,vllid loss 0.44323737338505764
1,train loss 0.2590239995292255,vllid loss 0.192740399863631
2,train loss 0.1492945881826537,vllid loss 0.1878271201132266
3,train loss 0.11162747205247615,vllid loss 0.15692275051798632
4,train loss 0.09241479550892452,vllid loss 0.14127378413198335
5,train loss 0.08345594648221594,vllid loss 0.12788715943448767
6,train loss 0.07677432749862217,vllid loss 0.15430622694510665
7,train loss 0.07334227925743858,vllid loss 0.1049750223984286
8,train loss 0.06893648486584425,vllid loss 0.13140174385876224
9,train loss 0.06270938281552249,vllid loss 0.11159493569259642
valid_score is0.9088249979284685


0,train loss 1.5018141750055503,vllid loss 0.46148253584741267
1,train loss 0.3044990985942979,vllid loss 0.2258655984400301
2,train loss 0.1726480471698261,vllid loss 0.238761557396301
3,train loss 0.12238698889001842,vllid loss 0.13442487266886058
4,train loss 0.10524507693323806,vllid loss 0.1275095752063608
5,train loss 0.09314087835062707,vllid loss 0.12948196382895213
6,train loss 0.0736751417978353,vllid loss 0.1539891346655686
7,train loss 0.06940467024819469,vllid loss 0.09781270348078881
8,train loss 0.0682717591117455,vllid loss 0.15461817248010887
9,train loss 0.061314811234706335,vllid loss 0.1057971764632619
valid_score is0.9139081283558795


0,train loss 1.9452132038029057,vllid loss 1.4371372371591546
1,train loss 0.5042292682951031,vllid loss 0.23535757353629477
2,train loss 0.17554925659274798,vllid loss 0.1977121618085651
3,train loss 0.13496630463116574,vllid loss 0.14852187064005642
4,train loss 0.11680848923112665,vllid loss 0.1808423092213249
5,train loss 0.1001815542006348,vllid loss 0.1386610390418159
6,train loss 0.08921401368358707,vllid loss 0.13122458029688605
7,train loss 0.08240300521877095,vllid loss 0.14224166249836323
8,train loss 0.07653170754766528,vllid loss 0.13716138880299303
9,train loss 0.07186226089889149,vllid loss 0.14094824069968215
valid_score is0.8938044002992067


0,train loss 1.5414221661913428,vllid loss 0.42136337211535824
1,train loss 0.24733147549259696,vllid loss 0.2482414518568104
2,train loss 0.14585630157887133,vllid loss 0.16951280649218167
3,train loss 0.11037071876609422,vllid loss 0.16288678902539336
4,train loss 0.09384430127564787,vllid loss 0.1340929188863224
5,train loss 0.08116168514737543,vllid loss 0.12950073208807597
6,train loss 0.07635934041898366,vllid loss 0.1232980043267905
7,train loss 0.06633343263918338,vllid loss 0.11672981553091367
8,train loss 0.06867502197984736,vllid loss 0.12085462598989452
9,train loss 0.06511982344844029,vllid loss 0.11002364600833345
valid_score is0.9033570839491402
----------------------------------------------------------------------------------------------------
CV mean score: 0.9037, std: 0.0071.
----------------------------------------------------------------------------------------------------
(20000, 10)


In [1]:
test_df=pd.DataFrame()
test_df["id"]=range(len(test_feat))
test_df["num_label"]=torch.argmax(torch.FloatTensor(predcit_results),dim=1).numpy()

NameError: ignored

In [65]:
test_df.head()

,id,num_label
0,0,7
1,1,9
2,2,2
3,3,2
4,4,7


In [66]:
def name2label_conv(data):
    name2label={
        "财经":0,"房产":1,"家居":2,"教育":3,"科技":4,"时尚":5,"时政":6,"游戏":7,"娱乐":8,"体育":9
    }
    data["num_label"]=data["class_label"].apply(lambda x: name2label[x])
    return data

In [67]:
def label2name_conv(data):
    label2name={0: '财经',
     1: '房产',
     2: '家居',
     3: '教育',
     4: '科技',
     5: '时尚',
     6: '时政',
     7: '游戏',
     8: '娱乐',
     9: '体育'}
    data["class_label"]=data["num_label"].apply(lambda x: label2name[x])
    return data

In [68]:
def label2level_conv(data):
    level={"财经":"高风险",
           "房产":"中风险",
           "家居":"可公开",
           "教育":"低风险",
           "科技":"中风险",
           "时尚":"低风险",
           "时政":"高风险",
           "游戏":"低风险",
           "娱乐":"可公开",
           "体育":"可公开"}
    data["rank_label"]=data["class_label"].apply(lambda x: level[x])
    return data

In [69]:
def label2level_conv(data):
    level={"财经":"高风险",
           "房产":"中风险",
           "家居":"可公开",
           "教育":"低风险",
           "科技":"中风险",
           "时尚":"低风险",
           "时政":"高风险",
           "游戏":"低风险",
           "娱乐":"可公开",
           "体育":"可公开"}
    data["rank_label"]=data["class_label"].apply(lambda x: level[x])
    return data

In [70]:
def submit(data,filename):
    data=label2name_conv(data)
    data=label2level_conv(data)
    data=data[["id","class_label","rank_label"]]
    data.to_csv(filename,index=False)

In [71]:
submit(test_df,"wavenet.csv")

In [1]:
import torch

In [2]:
a=torch.randn(10,10)

In [4]:
a[0]

tensor([ 0.8447, -1.6962, -1.3345,  1.3711,  0.1208, -1.0760, -1.5499, -1.5488,
         1.9907, -1.1830])

In [7]:
import  torch.nn.functional as F

In [72]:
def self_train(data_path,checkpoint_name,step,max_seq_len=256, model='bert-base-chinese', train_aug=False):

  #read data 
  tokenizer = BertTokenizer.from_pretrained(model)

  def text_filter(sentence: str) -> str:
        """
        过滤掉非汉字和标点符号和非数字
        :param sentence:
        :return:
        """
        line = sentence.replace('\n', '。')
        # 过滤掉非汉字和标点符号和非数字
        linelist = [word for word in line if
                    word >= u'\u4e00' and word <= u'\u9fa5' or word in ['，', '。', '？', '！',
                                                                        '：'] or word.isdigit()]
        return ''.join(linelist)
  def fliter_data(data):
      data['content']=data['content'].astype(str)
      data["text"]=data["content"].apply(lambda x:text_filter(x))
      return data 

  train_labeled=pd.read_csv(data_path+"/"+str(setp)+"train.csv")
  valid_labeled=pd.read_csv(data_path+"/valid.csv")
  train_ul=pd.read_csv(data_path+"/"+str(setp)+"train_ul.csv")
  test_df=pd.read_csv(data_path+"/"+str(setp)+"test_df.csv")



  train_labeled=fliter_data(train_labeled)
  valid_labeled=filter_data(valid_labeled)
  train_ul=fliter_data(train_ul)
  test_df=fliter_data(test_df)

  total_unlabeled=pd.concat([train_ul,test_df])

  train_labeled_dataset = loader_labeled(list(train_labeled["text"]),list(train_labeled["num_label"]), tokenizer, max_seq_len, train_aug)
  train_unlabeled_dataset = loader_unlabeled(list(total_unlabeled["text"]),tokenizer, max_seq_len, train_aug)
  val_dataset = loader_labeled(list(valid_labeled["text"]),list(valid_labeled["num_label"]), tokenizer, max_seq_len, train_aug)
  test_dataset = loader_unlabeled(list(test_df["text"]),tokenizer, max_seq_len, train_aug)

  # 制作loader
  labeled_trainloader = Data.DataLoader(
        dataset=train_labeled_set, batch_size=args.batch_size, shuffle=True)
  unlabeled_trainloader = Data.DataLoader(
        dataset=train_unlabeled_set, batch_size=args.batch_size_u, shuffle=True)
  ## use for self_training 
  unlabeled_trainloader2 = Data.DataLoader(
        dataset=train_unlabeled_set, batch_size=args.batch_size_u, shuffle=False)
  val_loader = Data.DataLoader(
        dataset=val_set, batch_size=20, shuffle=False)
  test_loader = Data.DataLoader(
        dataset=test_set, batch_size=20, shuffle=False)

  #read model from prev train 
  #checkpoint=torch.load(checkpoint_name)
  #model.load_state_dict(checkpoint)

  test_accs = []
  #再次训练
  for epoch in range(args.epochs):
        args.train_aug=False
        #调用train函数, 给定有标签数据，无标签数据，模型，优化器，损失函数，labels数量，是否数据增强
        train(labeled_trainloader, unlabeled_trainloader, model, optimizer,scheduler, train_criterion, epoch, n_labels, 256,args.train_aug)

        # scheduler.step()
        # _, train_acc = validate(labeled_trainloader, model,  criterion, epoch, mode='Train Stats')
        #print("epoch {}, train acc {}".format(epoch, train_acc))

        val_loss, val_acc = validate(val_loader, model, criterion, epoch, mode='Valid Stats')

        print("epoch {}, 验证集准确率 {}, 验证集损失 {}".format(epoch, val_acc, val_loss))

        print('Epoch: ', epoch)
        
  ## make predict 
  append_df=pd.DataFrame([])
  ## make a predict 
  outputs=[]
  for batch_idx, (inputs, length) in enumerate(test_loader):
            if n_gpu !=0:
                inputs = inputs.cuda()
            outputs.extend(model(inputs).detach().cpu().numpy())
  ## add the high probility sample to train_data

  ## 选择低熵样本
 with torch.no_grad():
    outputs_u=torch.tensor(outputs_u)
    #precict 
    _,test_df["num_label"]=torch.max(torch.tensor(outputs), 1)
    test_df["entropy"]=torch.sum(-F.softmax(outputs_u, dim=1) * F.log_softmax(outputs_u, dim=1), dim=1).cpu().numpy()
    ## select test_df
    test_df=test_df.sort_values("entropy").reset_index(drop=True)
    append_df1=test_df.iloc[:int(0.1 * len(test_df)),]
    test_df=test_df.iloc[int(0.1 * len(test_df)):,]
    append_df=pd.concat([append_df,append_df1])

  outputs=[]
  for batch_idx, (inputs, length) in enumerate(unlabeled_trainloader2):
            if n_gpu !=0:
                inputs = inputs.cuda()
            outputs.extend(model(inputs).detach().cpu().numpy())
  ## add the high probility sample to train_data

  ## 选择低熵样本
  with torch.no_grad():
    outputs_u=torch.tensor(outputs_u)
    #precict 
    _,train_ul["num_label"]=torch.max(torch.tensor(outputs), 1)
    train_ul["entropy"]=torch.sum(-F.softmax(outputs_u, dim=1) * F.log_softmax(outputs_u, dim=1), dim=1).cpu().numpy()
    ## select test_df
    train_ul=train_ul.sort_values("entropy").reset_index(drop=True)
    append_df2=train_ul.iloc[:int(0.1 * len(train_ul)),]
    train_ul=train_ul.iloc[int(0.1 * len(train_ul)):,]
    append_df=pd.concat([append_df,append_df2])

  ## get new train_labeled,train_ul,test_df 
  train_labeled=pd.concat([train_labeled,append_df])
  train_ul=train_ul
  test_df=test_df 

  train_labeled.to_csv(data_path+"/"+str(step+1)+"train.csv",index=False)
  train_ul.to_csv(data_path+"/"+str(step+1)+"train_ul.csv",index=False)
  test_df.to_csv(data_path+"/"+str(step+1)+"test_df.csv",index=False)

  torch.save(model.module.state_dict(), "/content/drive/My Drive/MixText/code/model/self_train_model"+str(step)+".pt")
  

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 87)